In [26]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Josué e Rose\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [27]:
# abrir a página index (entrar no site da busca jurídica)
import os

caminho = os.getcwd()
arquivo = caminho + r"\index.html"

In [28]:
# importar a base de dados
import pandas as pd

In [29]:
processos_df = pd.read_excel('Processos.xlsx')

display(processos_df)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [30]:
from selenium.webdriver import ActionChains

In [31]:
import time

In [32]:
navegador.get(arquivo)
# para cada processo (linha da tabela)
for linha in processos_df.index:

    # abrir a lista de cidades
    ActionChains(navegador).move_to_element(navegador.find_element(By.CLASS_NAME, 'dropdown-menu')).perform()

    # selecionando o cidade
    cidade = processos_df.loc[linha, 'Cidade']
    print(cidade)
    
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

    aba_original = navegador.window_handles[0]
    nova_aba = navegador.window_handles[1]
    # mudar para a nova aba
    navegador.switch_to.window(nova_aba)

    # preencher o formulário com os dados de busca
    navegador.find_element(By.ID, 'nome').send_keys(str(processos_df.loc[linha,'Nome']))

    navegador.find_element(By.ID, 'advogado').send_keys(str(processos_df.loc[linha, 'Advogado']))

    navegador.find_element(By.ID, 'numero').send_keys(str(processos_df.loc[linha, 'Processo']))
    # clicar em pesquisar

    navegador.find_element(By.CLASS_NAME,'registerbtn').click()
    #  confirmar a pesquisa

    alerta = navegador.switch_to.alert
    alerta.accept()

    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)

    texto_alerta = alerta.text

    if 'Processo encontrado com sucesso' in texto_alerta:
        processos_df.loc[linha, 'Status'] = 'ENCONTRADO'
        alerta.accept()
    else:
        processos_df.loc[linha, 'Status'] = 'NÃO ENCONTRADO'
        alerta.accept()
    # esperar o resultado da pesquisa e agir de acordo com o resultado
    navegador.close()

    navegador.switch_to.window(aba_original)


Distrito Federal
Rio de Janeiro
Rio de Janeiro
São Paulo


In [33]:
display(processos_df)
processos_df.to_excel('Processos atualizados.xlsx')

navegador.quit()

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NÃO ENCONTRADO
1,João,Lawyer Alon,EB3792,Rio de Janeiro,ENCONTRADO
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NÃO ENCONTRADO
3,Carol,Amanda,PC5197,São Paulo,NÃO ENCONTRADO
